# 瓶装酒瑕疵数据清洗

## 读入数据

In [1]:
import os
import json
DATASET_PATH = './data/chongqing1_round1_train1_20191223'
with open(os.path.join(DATASET_PATH, 'annotations.json')) as f:
    json_file = json.load(f)
    
print('所有图片的数量：', len(json_file['images']))
print('所有标注的数量：', len(json_file['annotations']))

所有图片的数量： 4516
所有标注的数量： 6945


## 找到和背景有关的图片

In [2]:
bg_imgs = set()  # 所有标注中包含背景的图片 id
for c in json_file['annotations']:
    if c['category_id'] == 0:
        bg_imgs.add(c['image_id'])
        
print('所有标注中包含背景的图片数量：', len(bg_imgs))

bg_only_imgs = set()  # 只有背景的图片的 id
for img_id in bg_imgs:
    co = 0
    for c in json_file['annotations']:
        if c['image_id'] == img_id:
            co += 1
    if co == 1:
        bg_only_imgs.add(img_id)
print('只包含背景的图片数量：', len(bg_only_imgs))

所有标注中包含背景的图片数量： 1168
只包含背景的图片数量： 1145


## 删除只有背景标注的图片

In [3]:
images_to_be_deleted = []
for img in json_file['images']:
    if img['id'] in bg_only_imgs:
        images_to_be_deleted.append(img)
# 删除的是只有一个标注，且为 background 的的图片
print('待删除图片的数量：', len(images_to_be_deleted))
for img in images_to_be_deleted:
    json_file['images'].remove(img)


print('处理之后图片的数量：', len(json_file['images']))

待删除图片的数量： 1145
处理之后图片的数量： 3371


注：这里有一张图片有两个 annotation，全是 background，所以删漏掉了。正确处理后处理之后数量为 3370。这个数据 coco api 会自动处理。

## 删除所有关于背景的标注

In [4]:
ann_to_be_deleted = []
for c in json_file['annotations']:
    if c['category_id'] == 0:
        ann_to_be_deleted.append(c)
        
print('待删除标注的数量：', len(ann_to_be_deleted))
for img in ann_to_be_deleted:
    json_file['annotations'].remove(img)

print('处理之后标注的数量：', len(json_file['annotations']))

待删除标注的数量： 1170
处理之后标注的数量： 5775


## 删除 categories 中关于背景的部分

In [5]:
bg_cate = {'supercategory': '背景', 'id': 0, 'name': '背景'}
json_file['categories'].remove(bg_cate)
json_file['categories']

[{'supercategory': '瓶盖破损', 'id': 1, 'name': '瓶盖破损'},
 {'supercategory': '喷码正常', 'id': 9, 'name': '喷码正常'},
 {'supercategory': '瓶盖断点', 'id': 5, 'name': '瓶盖断点'},
 {'supercategory': '瓶盖坏边', 'id': 3, 'name': '瓶盖坏边'},
 {'supercategory': '瓶盖打旋', 'id': 4, 'name': '瓶盖打旋'},
 {'supercategory': '瓶盖变形', 'id': 2, 'name': '瓶盖变形'},
 {'supercategory': '标贴气泡', 'id': 8, 'name': '标贴气泡'},
 {'supercategory': '标贴歪斜', 'id': 6, 'name': '标贴歪斜'},
 {'supercategory': '喷码异常', 'id': 10, 'name': '喷码异常'},
 {'supercategory': '标贴起皱', 'id': 7, 'name': '标贴起皱'}]

## 标注的 id 有重复的，这里重新标号

In [6]:
for idx in range(len(json_file['annotations'])):
    json_file['annotations'][idx]['id'] = idx

## 导出数据

In [7]:
with open(os.path.join(DATASET_PATH, 'annotations_washed.json'), 'w') as f:
    json.dump(json_file, f)